# Import des modules

In [376]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

#Selection
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate, KFold, StratifiedKFold
from sklearn.inspection import permutation_importance
from sklearn.utils.class_weight import compute_sample_weight
import shap

#Modèles
from sklearn.dummy import DummyClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

#Metriques
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report, precision_recall_curve, average_precision_score, roc_curve, auc

In [377]:
fc = pd.read_csv('fc_after_feature_engineering.csv')
print(fc.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 48 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   a_quitte_l_entreprise                      1470 non-null   int64  
 1   age                                        1470 non-null   int64  
 2   annees_dans_l_entreprise                   1470 non-null   int64  
 3   annees_dans_le_poste_actuel                1470 non-null   int64  
 4   annees_depuis_la_derniere_promotion        1470 non-null   int64  
 5   annees_experience_totale                   1470 non-null   int64  
 6   annes_sous_responsable_actuel              1470 non-null   int64  
 7   augmentation_salaire_precedente            1470 non-null   int64  
 8   departement_Commercial                     1470 non-null   int64  
 9   departement_Consulting                     1470 non-null   int64  
 10  departement_Ressources H

# Déclaration des Dataframes X et y
- Un DataFrame contenant les features => X
- Un Pandas Series contenant la colonne cible => y

In [378]:
#Un homme , Célibataire , Entre 30 et 40 ans , Consultant , Entre 1 et 7 années dans l’entreprise, Un revenu compris entre 2500€ et 6000€, Domaine d’etude Infra & Cloud, Qui a moins de 5 années sous son responsable actuel, Qui a une distance domicile travail entre 3 et 17km

# Test 1
#columns_base = ['genre', 'statut_marital', 'age', 'annees_dans_l_entreprise', 'revenu_mensuel', 'distance_domicile_travail', 'satisfaction_globale']
# Test 2
columns_base = ['genre', 'revenu_mensuel', 'age', 'annees_dans_l_entreprise', 'distance_domicile_travail',
                'score_salaire', 'satisfaction_globale']

columns_domaine_etude = [col for col in fc.columns if col.startswith('domaine_etude')]
columns_poste = [col for col in fc.columns if col.startswith('poste')]
columns_statut_marital = [col for col in fc.columns if col.startswith('statut_marital')]
columns_satisfaction = [col for col in fc.columns if col.startswith('satisfaction')]

all_columns = columns_base + columns_statut_marital + columns_domaine_etude + columns_poste + columns_satisfaction

X = fc[all_columns]
y = fc['a_quitte_l_entreprise']

In [379]:
def train_model(model, X_train, X_test, y_train, y_test, model_name):
    """
    Entraîne un modèle et retourne les métriques train/test avec détection d'overfitting.

    Args:
        model: Le modèle à entraîner
        X_train, X_test: Features d'entraînement et de test
        y_train, y_test: Labels d'entraînement et de test
        model_name: Nom du modèle (string)

    Returns:
        tuple: (results, report_test, cm, curves, pipeline)
    """
    # Poids
    weights = compute_sample_weight(class_weight='balanced', y=y_train)

    # Création d'un pipeline avec standardisation puis modèle
    tt_pipeline = Pipeline(steps=[
        ('scaler', StandardScaler()),
        ('model', model)
    ])

    # Entraînement avec les poids
    tt_pipeline.fit(X_train, y_train, model__sample_weight=weights)

    # Prédictions sur TRAIN
    y_pred_train = tt_pipeline.predict(X_train)
    class_report_train = classification_report(y_train, y_pred_train, output_dict=True)

    # Prédictions sur TEST
    y_pred_test = tt_pipeline.predict(X_test)
    class_report_test = classification_report(y_test, y_pred_test, output_dict=True)

    # Matrice de confusion (test)
    tt_cm = confusion_matrix(y_test, y_pred_test)

    # Calcul des écarts (train - test) pour détecter l'overfitting
    accuracy_gap = class_report_train['accuracy'] - class_report_test['accuracy']
    f1_gap = class_report_train['macro avg']['f1-score'] - class_report_test['macro avg']['f1-score']

    # Indicateur d'overfitting (seuil à 12% d'écart)
    overfitting_flag = 'OUI' if (accuracy_gap > 0.12 or f1_gap > 0.12) else 'NON'

    # Extraction des métriques principales
    tt_results = {
        'model': model_name,
        'method': 'train_test',
        # Métriques TRAIN
        'train_accuracy': round(class_report_train['accuracy'], 2),
        'train_f1_macro': round(class_report_train['macro avg']['f1-score'], 2),
        'train_precision_macro': round(class_report_train['macro avg']['precision'], 2),
        'train_recall_macro': round(class_report_train['macro avg']['recall'], 2),
        # Métriques TEST
        'test_accuracy': round(class_report_test['accuracy'], 2),
        'test_f1_macro': round(class_report_test['macro avg']['f1-score'], 2),
        'test_precision_macro': round(class_report_test['macro avg']['precision'], 2),
        'test_recall_macro': round(class_report_test['macro avg']['recall'], 2),
        # Écarts et overfitting
        'accuracy_gap': round(accuracy_gap, 2),
        'f1_gap': round(f1_gap, 2),
        'overfitting': overfitting_flag,
        'confusion_matrix': str(tt_cm.tolist()),
    }

    # Récupérer le modèle entraîné du pipeline
    trained_model = tt_pipeline.named_steps['model']
    X_test_scaled = tt_pipeline.named_steps['scaler'].transform(X_test)

    # Vérifier si le modèle supporte predict_proba
    try:
        y_pred_proba = trained_model.predict_proba(X_test_scaled)
    except AttributeError:
        y_pred_proba = None

    # Données pour les courbes ROC et PR
    tt_curves = {
        'model': model_name,
        'y_true': y_test.copy() if hasattr(y_test, 'copy') else list(y_test),
        'y_pred_proba': y_pred_proba,
        'y_pred': y_pred_test
    }

    return tt_results, class_report_test, tt_cm, tt_curves, tt_pipeline

In [380]:
def compute_permutation_importance(pipeline, X_test, y_test, model_name, n_repeats=10):
    """
    Calcule la permutation importance pour un modèle entraîné.

    Args:
        pipeline: Pipeline entraîné contenant scaler et modèle
        X_test: Features de test
        y_test: Labels de test
        model_name: Nom du modèle
        n_repeats: Nombre de répétitions pour la permutation (défaut: 10)

    Returns:
        pd.DataFrame: DataFrame avec les importances par feature
    """
    trained_model = pipeline.named_steps['model']
    X_test_scaled = pipeline.named_steps['scaler'].transform(X_test)

    # Permutation importance sur les données scalées
    pi = permutation_importance(
        trained_model,
        X_test_scaled,
        y_test,
        n_repeats=n_repeats,
        random_state=82
    )

    # Construction du DataFrame d'importances
    df_importances = pd.DataFrame({
        'model': model_name,
        'feature': X_test.columns,
        'importance_mean': pi.importances_mean,
        'importance_std': pi.importances_std
    })

    # Ajouter les feature importances natives si disponibles
    trained_model = pipeline.named_steps['model']
    if hasattr(trained_model, 'feature_importances_'):
        df_importances['feature_importances'] = trained_model.feature_importances_

    return df_importances

In [381]:
def compute_shap_values(pipeline, X_test, model_name):
    """
    Calcule les valeurs SHAP pour un modèle tree-based.

    Args:
        pipeline: Pipeline entraîné contenant scaler et modèle
        X_test: Features de test
        model_name: Nom du modèle

    Returns:
        dict: Dictionnaire contenant les valeurs SHAP et métadonnées
              ou None si le modèle n'est pas compatible
    """
    trained_model = pipeline.named_steps['model']
    X_test_scaled = pipeline.named_steps['scaler'].transform(X_test)

    # SHAP uniquement pour modèles tree-based
    tree_based_models = (XGBClassifier,CatBoostClassifier)

    if not isinstance(trained_model, tree_based_models):
        print(f"⚠ Modèle ignoré (non compatible avec TreeExplainer): {type(trained_model).__name__}")
        return None

    # Calcul des valeurs SHAP
    explainer = shap.TreeExplainer(trained_model)
    shap_values = explainer.shap_values(X_test_scaled)

    # Récupérer les noms de features
    feature_names_in = (
        trained_model.feature_names_in_
        if hasattr(trained_model, 'feature_names_in_')
        else X_test.columns.tolist()
    )

    # Métadonnées SHAP
    shap_metadata = {
        'model': model_name,
        'shap_values': shap_values,
        'X_test': X_test,
        'X_test_scaled': X_test_scaled,
        'feature_names_in': feature_names_in
    }

    return shap_metadata

In [382]:
def cross_validation_evaluation(model, X, y, model_name, cv=5):
    """
    Effectue une validation croisée et retourne les métriques avec détection d'overfitting.

    Args:
        model: Le modèle à évaluer
        X: Features complètes
        y: Labels complets
        model_name: Nom du modèle (string)
        cv: Nombre de folds (défaut: 5)

    Returns:
        dict: Dictionnaire contenant les résultats moyens avec scores train et test
    """

    # Création du pipeline avec scaling
    pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', model)
    ])

    # Définition des métriques à calculer
    scoring = ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']

    # Validation croisée avec return_train_score=True pour détecter overfitting
    results_cross_validate = cross_validate(pipeline, X, y, cv=cv, scoring=scoring, return_train_score=True, error_score='raise')

    # Calcul des écarts moyens (train - test)
    accuracy_gap_cv = results_cross_validate['train_accuracy'].mean() - results_cross_validate['test_accuracy'].mean()
    f1_gap_cv = results_cross_validate['train_f1_macro'].mean() - results_cross_validate['test_f1_macro'].mean()

    # Indicateur d'overfitting
    overfitting_flag_cv = 'OUI' if (accuracy_gap_cv >= 0.12 or f1_gap_cv >= 0.12) else 'NON'

    # Calcul des moyennes et écarts-types
    cv_results = {
        'model': model_name,
        'method': 'cross_validation',
        # Métriques TRAIN
        'train_accuracy': round(results_cross_validate['train_accuracy'].mean(), 2),
        'train_accuracy_std': round(results_cross_validate['train_accuracy'].std(), 2),
        'train_f1_macro': round(results_cross_validate['train_f1_macro'].mean(), 2),
        'train_f1_macro_std': round(results_cross_validate['train_f1_macro'].std(), 2),
        'train_precision_macro': round(results_cross_validate['train_precision_macro'].mean(), 2),
        'train_recall_macro': round(results_cross_validate['train_recall_macro'].mean(), 2),
        # Métriques TEST
        'test_accuracy': round(results_cross_validate['test_accuracy'].mean(), 2),
        'test_accuracy_std': round(results_cross_validate['test_accuracy'].std(), 2),
        'test_f1_macro': round(results_cross_validate['test_f1_macro'].mean(), 2),
        'test_f1_macro_std': round(results_cross_validate['test_f1_macro'].std(), 2),
        'test_precision_macro': round(results_cross_validate['test_precision_macro'].mean(), 2),
        'test_recall_macro': round(results_cross_validate['test_recall_macro'].mean(), 2),
        # Écarts et overfitting
        'accuracy_gap': round(accuracy_gap_cv, 2),
        'f1_gap': round(f1_gap_cv, 2),
        'overfitting': overfitting_flag_cv,
        'confusion_matrix': 'N/A',
    }

    return cv_results

In [383]:
random_state = 82

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state, stratify=y)

# Définition des modèles
models = {
    'DummyClassifier': DummyClassifier(
        strategy='most_frequent',
        random_state=random_state),

    'XGBClassifier': XGBClassifier(
        n_estimators=100,
        max_depth=3,
        learning_rate=0.1,     # Plus petit = apprentissage plus lent
        subsample=0.8,          # 80% des données par arbre
        reg_lambda=2,         # Régularisation L2
        eval_metric='logloss',
        scale_pos_weight=5,
        random_state=random_state),

    'CatBoostClassifier': CatBoostClassifier(
        iterations=300,
        depth=4,
        learning_rate=0.05,
        l2_leaf_reg=3,        # Régularisation
        verbose=False,
        random_state=random_state)
}


results_prov = []
results_global = []
class_report = []
all_curves = []
all_df_importances = []
all_importances_metadata = []

for model_name, model in models.items():
    print(f">>> Évaluation de {model_name}...")

    # Entraînement
    tt_results, class_report_test, tt_cm, tt_curves, tt_pipeline = train_model(model, X_train, X_test, y_train, y_test, model_name)

    # Cross-validation
    cv_results = cross_validation_evaluation(model, X, y, model_name, cv=5)

    # Permutation importance
    df_importances = compute_permutation_importance(tt_pipeline, X_test, y_test, model_name)

    # SHAP
    shap_metadata = compute_shap_values(tt_pipeline, X_test, model_name)

    # Formater les métadonnées pour garder la compatibilité
    importances_metadata = {
        'model': model_name,
        'X_test': X_test,
        'X_test_scaled': tt_pipeline.named_steps['scaler'].transform(X_test),
        'feature_names_in': X_test.columns.tolist(),
    }

    class_report.append({
            'model': model_name,
            'method': 'train_test',
            'report': class_report_test,
            'confusion_matrix': tt_cm
        })

    # Sauvegarde des resultats
    results_prov.append(tt_results)
    results_prov.append(cv_results)

    # Conversion en DataFrame
    results_global = pd.DataFrame(results_prov)
    # Réorganisation des colonnes pour la lisibilité
    cols_order = ['model', 'method','train_accuracy', 'test_accuracy', 'accuracy_gap',
                  'train_f1_macro', 'test_f1_macro', 'f1_gap','overfitting',
                  'train_precision_macro', 'test_precision_macro','train_recall_macro', 'test_recall_macro']
    # Ajout des colonnes std si elles existent
    std_cols = [col for col in results_global.columns if '_std' in col]
    cols_order.extend(std_cols)
    cols_order.append('confusion_matrix')
    # Colonnes présentes dans le DataFrame
    cols_order = [col for col in cols_order if col in results_global.columns]
    results_global = results_global[cols_order]

    all_curves.append(tt_curves)
    all_df_importances.append(df_importances)
    all_importances_metadata.append(importances_metadata)



>>> Évaluation de DummyClassifier...
⚠ Modèle ignoré (non compatible avec TreeExplainer): DummyClassifier
>>> Évaluation de XGBClassifier...


/Users/tico/Documents/GIT/PredictDataBuilding/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Users/tico/Documents/GIT/PredictDataBuilding/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Users/tico/Documents/GIT/PredictDataBuilding/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this b

>>> Évaluation de CatBoostClassifier...


# Sauvegardes des résultats

In [ ]:
# Fichier principal avec toutes les métriques
results_global.to_csv(f'exports/classification_summary.csv', index=False)
print(f"\n✓ Résumé sauvegardé: classification_summary.csv")

# Fichier avec les rapports détaillés de classification
detailed_data = []
for item in class_report:
    model = item['model']
    report = item['report']

    # Extraction des métriques par classe
    for class_label, metrics in report.items():
        if class_label not in ['accuracy', 'macro avg', 'weighted avg']:
            detailed_data.append({
                'model': model,
                'class': class_label,
                'precision': round(metrics['precision'], 2),
                'recall': round(metrics['recall'], 2),
                'f1-score': round(metrics['f1-score'], 2),
                'support': metrics['support']
            })

detailed_df = pd.DataFrame(detailed_data)
detailed_df.to_csv(f'exports/classification_by_class.csv', index=False)
print(f"✓ Résultats par classe sauvegardés: classification_by_class.csv")

# Génération des graphiques

In [ ]:
def _plot_binary_roc_curves(all_curves):
    """
    Courbes ROC pour classification binaire.
    """
    plt.figure(figsize=(10, 8))

    for data in all_curves:
        model_name = data['model']
        y_true = data['y_true']

        # Vérifier si les probabilités sont disponibles
        if 'y_pred_proba' in data and data['y_pred_proba'] is not None:
            y_score = data['y_pred_proba']
            # Si format (n_samples, n_classes), prendre la colonne de la classe positive
            if len(y_score.shape) > 1:
                y_score = y_score[:, 1]
        else:
            print(f"⚠️  Pas de probabilités pour {model_name}, utilisation des prédictions")
            y_score = data['y_pred']

        # Calculer la courbe ROC
        fpr, tpr, _ = roc_curve(y_true, y_score)
        roc_auc = auc(fpr, tpr)

        plt.plot(fpr, tpr, lw=2, label=f'{model_name} (AUC = {roc_auc:.3f})')

    # Ligne diagonale (classificateur aléatoire)
    plt.plot([0, 1], [0, 1], 'k--', lw=2, label='Aléatoire (AUC = 0.500)')

    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Taux de Faux Positifs', fontsize=12)
    plt.ylabel('Taux de Vrais Positifs', fontsize=12)
    plt.title('Courbes ROC - Comparaison des Modèles', fontsize=14, fontweight='bold')
    plt.legend(loc="lower right", fontsize=10)
    plt.grid(alpha=0.3)
    plt.tight_layout()

    filename = f'images/classification_roc_curves.png'
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"✓ Courbes ROC sauvegardées: classification_roc_curves.png")

def _plot_binary_pr_curves(all_curves):
    """
    Courbes Précision-Rappel pour classification binaire.
    """
    plt.figure(figsize=(10, 8))

    for data in all_curves:
        model_name = data['model']
        y_true = data['y_true']

        if 'y_pred_proba' in data and data['y_pred_proba'] is not None:
            y_score = data['y_pred_proba']
            if len(y_score.shape) > 1:
                y_score = y_score[:, 1]
        else:
            print(f"⚠️  Pas de probabilités pour {model_name}, utilisation des prédictions")
            y_score = data['y_pred']

        precision, recall, _ = precision_recall_curve(y_true, y_score)
        avg_precision = average_precision_score(y_true, y_score)

        plt.plot(recall, precision, lw=2,
                 label=f'{model_name} (AP = {avg_precision:.3f})')

    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Rappel', fontsize=12)
    plt.ylabel('Précision', fontsize=12)
    plt.title('Courbes Précision-Rappel - Comparaison des Modèles',
              fontsize=14, fontweight='bold')
    plt.legend(loc="lower left", fontsize=10)
    plt.grid(alpha=0.3)
    plt.tight_layout()

    filename = f'images/classification_precision_recall_curves.png'
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"✓ Courbes Précision-Rappel sauvegardées: classification_precision_recall_curves.png")

def _plot_permutation_importance(all_df_importances):
    """
    Plot de la permutation importance pour tous les modèles.
    """
    plt.figure(figsize=(14, 8))

    # Définir une palette de couleurs pour différencier les modèles
    colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A', '#96CEB4', '#FFEAA7']

    all_df_importances = [df for df in all_df_importances
                      if not df['importance_mean'].isna().all() and (df['importance_mean'] != 0).any()]

    # Récupérer les features du premier DataFrame et trier alphabétiquement
    first_df = all_df_importances[0].copy()
    features = sorted(first_df['feature'].tolist())

    # Position des barres pour chaque modèle
    bar_height = 0.8 / len(all_df_importances)  # Ajuster la hauteur selon le nombre de modèles
    y_positions = range(len(features))

    for idx, df in enumerate(all_df_importances):
        # Trier le DataFrame selon l'ordre des features
        df_sorted = df.set_index('feature').loc[features].reset_index()

        # Récupérer le nom du modèle si disponible
        model_name = df.get('model', [f'Modèle {idx+1}'])[0] if 'model' in df.columns else f'Modèle {idx+1}'

        # Décaler chaque modèle verticalement
        y_offset = [y + idx * bar_height for y in y_positions]

        plt.barh(y_offset, df_sorted['importance_mean'],
                 height=bar_height,
                 label=model_name,
                 color=colors[idx % len(colors)],
                 alpha=0.8)

    # Ajuster les positions des labels sur l'axe Y
    plt.yticks([y + bar_height * (len(all_df_importances) - 1) / 2 for y in y_positions], features)
    plt.xlabel('Importance Mean', fontsize=12)
    plt.ylabel('Features', fontsize=12)
    plt.title(f'Permutation Importance - Comparaison des Modèles (classification)',
              fontsize=14, fontweight='bold')
    plt.legend(loc="best", fontsize=10)
    plt.grid(axis='x', alpha=0.3)
    plt.tight_layout()

    filename = f'images/classification_permutation_importance_comparison.png'
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"✓ Graphique de comparaison sauvegardé: {filename}")

def _plot_native_feature_importance(all_df_importances, all_importances_metadata):
    """
    Plot de la feature importance native pour les modèles tree-based,
    avec normalisation de chaque modèle pour les mettre sur la même échelle.
    """

    # Filtrer uniquement les modèles qui ont des feature_importances
    valid_data = [
        (df.copy(), meta) for df, meta in zip(all_df_importances, all_importances_metadata)
        if 'feature_importances' in df.columns and df['feature_importances'].notna().any()
    ]

    if not valid_data:
        print(f"⚠ Aucun modèle avec feature_importances natives trouvé. Graphique ignoré.")
        return

    plt.figure(figsize=(14, 8))

    # Palette de couleurs pour différencier les modèles
    colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A', '#96CEB4', '#FFEAA7']

    # Normalisation des importances pour chaque modèle (somme = 1)
    for df, _ in valid_data:
        total = df['feature_importances'].sum()
        if total != 0:
            df['feature_importances'] = df['feature_importances'] / total

    # Récupérer la liste ordonnée des features du premier modèle
    first_df = valid_data[0][0].sort_values(by='feature_importances', ascending=True)
    features = first_df['feature'].tolist()

    # Position verticale des barres
    bar_height = 0.8 / len(valid_data)
    y_positions = range(len(features))

    for idx, (df, meta) in enumerate(valid_data):
        # Trier selon l’ordre des features du premier modèle
        df_sorted = df.set_index('feature').loc[features].reset_index()

        # Nom du modèle
        model_name = df.get('model', [f'Modèle {idx + 1}'])[0] if 'model' in df.columns else f'Modèle {idx + 1}'

        # Décalage vertical pour éviter le chevauchement
        y_offset = [y + idx * bar_height for y in y_positions]

        plt.barh(
            y_offset,
            df_sorted['feature_importances'],
            height=bar_height,
            label=model_name,
            color=colors[idx % len(colors)],
            alpha=0.8
        )

    # Mise en forme
    plt.yticks([y + bar_height * (len(valid_data) - 1) / 2 for y in y_positions], features)
    plt.xlabel('Feature Importance (normalisée)', fontsize=12)
    plt.ylabel('Features', fontsize=12)
    plt.title(f'Native Feature Importance Normalisée - Comparaison des Modèles (classification)',
              fontsize=14, fontweight='bold')
    plt.legend(loc="best", fontsize=10)
    plt.grid(axis='x', alpha=0.3)
    plt.tight_layout()

    filename = f'images/classification_native_feature_importance_comparison_normalized.png'
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"✓ Graphique normalisé sauvegardé : {filename}")

def _plot_shap(all_importances_metadata):
    """
    Plot des valeurs SHAP pour les modèles tree-based.

    Args:
        all_importances_metadata: Liste de dictionnaires contenant X_test et shap_values
        output_prefix: Préfixe pour les noms de fichiers
    """
    for idx, meta in enumerate(all_importances_metadata):
        if meta['shap_values'] is None:
            continue

        model_name = meta.get('model', f'Modèle {idx + 1}')
        X_test = meta['X_test']
        shap_values = meta['shap_values']

        # Summary plot
        plt.figure(figsize=(12, 8))
        shap.summary_plot(shap_values, X_test, show=False)
        plt.title(f'SHAP Summary Plot - {model_name}', fontsize=14, fontweight='bold')
        plt.tight_layout()

        filename = f'images/classification_shap_summary_{model_name}.png'
        plt.savefig(filename, dpi=300, bbox_inches='tight')
        plt.close()
        print(f"✓ SHAP summary plot sauvegardé: {filename}")

def _plot_shap_waterfall_combined(all_importances_metadata, n_samples=5):
    """
    Plot combiné des waterfall SHAP pour plusieurs individus sur une seule figure.
    """
    for idx, meta in enumerate(all_importances_metadata):
        shap_plots = meta.get('shap_plots', [])

        if not shap_plots:
            continue

        model_name = meta.get('model', f'Modèle {idx + 1}')
        n_plots = min(n_samples, len(shap_plots))

        # Créer une figure avec subplots
        fig, axes = plt.subplots(n_plots, 1, figsize=(12, 5 * n_plots))
        if n_plots == 1:
            axes = [axes]

        for i in range(n_plots):
            plot_data = shap_plots[i]
            plt.sca(axes[i])

            shap.plots.waterfall(plot_data['explanation'], show=False)

            true_label = plot_data['true_label']
            pred_label = plot_data['predicted_label']
            individual_idx = plot_data['index']
            match_status = "✓" if true_label == pred_label else "✗"

            axes[i].set_title(
                f'Individu {individual_idx} - Vraie: {true_label} | Prédite: {pred_label} {match_status}',
                fontsize=10, fontweight='bold'
            )

        fig.suptitle(f'SHAP Waterfall Plots - {model_name}', fontsize=14, fontweight='bold', y=1.001)
        plt.tight_layout()

        filename = f'images/classification_{model_name}_waterfall_combined.png'
        plt.savefig(filename, dpi=300, bbox_inches='tight')
        plt.close()
        print(f"✓ SHAP waterfall combiné sauvegardé: {filename}")

_plot_binary_roc_curves(all_curves)
_plot_binary_pr_curves(all_curves)
_plot_permutation_importance(all_df_importances)
_plot_native_feature_importance(all_df_importances, all_importances_metadata)
_plot_shap(all_importances_metadata)
#_plot_shap_waterfall_combined(all_importances_metadata)

# Optimisation des Hypers-Parametres

In [ ]:
def get_param_grids():
    """Définit les grilles de paramètres pour chaque modèle."""
    return {
        'DummyClassifier': {
            'strategy': ['most_frequent', 'stratified', 'uniform']
        },
        'RandomForestClassifier': {
            'n_estimators': [100, 200, 300],
            'max_depth': [3, 4, 5, 6, 7, None],
            'min_samples_split': [2, 5, 10, 15],
            'min_samples_leaf': [1, 5, 10],
            'max_features': ['sqrt', 'log2']
        },
        'XGBClassifier': {
            'n_estimators': [100, 200, 300],
            'max_depth': [3, 5, 7],
            'learning_rate': [0.01, 0.1, 0.3],
            'subsample': [0.8, 1.0],
            'colsample_bytree': [0.8, 1.0],
            'reg_alpha': [0, 0.1, 1],
            'reg_lambda': [1, 2, 5]
        },
        'CatBoostClassifier': {
            'iterations': [100, 200, 300],
            'depth': [4, 6, 8],
            'learning_rate': [0.01, 0.05, 0.1],
            'l2_leaf_reg': [1, 3, 5],
            'subsample': [0.8, 1.0]
        }
    }


def perform_grid_search(model, param_grid, X_train, y_train, cv=5, scoring='f1_macro'):
    """Effectue un GridSearchCV pour un modèle donné."""
    cv_strategy = StratifiedKFold(n_splits=cv, shuffle=True, random_state=42)

    grid_search = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        cv=cv_strategy,
        scoring=scoring,
        n_jobs=-1,
        verbose=0
    )

    grid_search.fit(X_train, y_train)
    return grid_search


def compare_models_gridsearch(X, y, test_size=0.15, cv=5, scoring='f1_macro', random_state=82):
    """Compare tous les modèles avec GridSearchCV."""
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state, stratify=y
    )

    param_grids = get_param_grids()

    base_models = {
        'DummyClassifier': DummyClassifier(random_state=random_state),
        'XGBClassifier': XGBClassifier(random_state=random_state, eval_metric='logloss'),
        'CatBoostClassifier': CatBoostClassifier(random_state=random_state, verbose=False)
    }

    results = []

    for model_name, base_model in base_models.items():
        grid_search = perform_grid_search(
            base_model,
            param_grids[model_name],
            X_train,
            y_train,
            cv=cv,
            scoring=scoring
        )

        best_model = grid_search.best_estimator_
        y_pred_test = best_model.predict(X_test)
        report_test = classification_report(y_test, y_pred_test, output_dict=True)

        result = {
            'model': model_name,
            'best_cv_score': round(grid_search.best_score_, 4),
            'test_accuracy': round(report_test['accuracy'], 4),
            'test_f1_macro': round(report_test['macro avg']['f1-score'], 4),
            'test_precision': round(report_test['macro avg']['precision'], 4),
            'test_recall': round(report_test['macro avg']['recall'], 4),
            'best_params': str(grid_search.best_params_)
        }
        results.append(result)

    return pd.DataFrame(results)


# Exécution
results_df = compare_models_gridsearch(
    X, y,
    test_size=0.15,
    cv=5,
    scoring='f1_macro',
    random_state=82
)

# Sauvegarde
results_df.to_csv('exports/gridsearch_best_results.csv', index=False)